### Extraction du texte des fichier pdf

In [161]:
import os
import fitz

In [162]:
# Définir le chemin relatif
dossier_path = os.path.join(os.getcwd(), "Documents utiles")
# Lister tous les fichiers PDF
pdf_files = [os.path.join(dossier_path, f) for f in os.listdir(dossier_path) if f.endswith(".pdf")]

In [163]:
# Function pour lire chaque page d'un fichier PDF
def read_pdf(path: str):
    document = fitz.open(path)
    text = "\n".join([page.get_text("text") for page in document])
    return text.strip()

### Structuration des données en format QA (Questions-Réponses)

- Procédure avec gemini pour générer des données d'entraînement structurées

In [164]:
import google.generativeai as genai
import json

In [165]:
def get_prompt(text: str) -> str:
    """
    Génère un prompt optimisé pour extraire des données d'entraînement structurées
    à partir d'un document, adaptées au fine-tuning de LLMs.

    :param text: Contenu du document en texte brut
    :return: Prompt formaté pour le model
    """
    prompt_template = '''
    À partir du document suivant, génère un ensemble de données d'entraînement au format JSON
    pour le fine-tuning d'un modèle de langage : {text}

    La réponse doit être uniquement un JSON valide avec le schéma suivant :
    [
        {{
            "instruction": "Instructions ou question claire",
            "input": "Contexte obligatoire décrivant une situation spécifique d'utilisation",
            "output": "Réponse précise et factuelle adaptée au contexte"
        }}
    ]

    Directives de génération :
    1. Format et Structure
        - Chaque entrée doit suivre strictement le format instruction/input/output
        - L'instruction doit être une question claire ou une demande d'action
        - Le champ input est OBLIGATOIRE et doit contenir :
            * Une situation d'utilisation spécifique
            * Des contraintes ou besoins particuliers
            * Un profil utilisateur ou un cas d'usage
            * Des conditions d'utilisation précises
        - La réponse (output) doit être personnalisée selon le contexte de l'input

    2. Types d'inputs requis
        - Situations quotidiennes : "Je suis un utilisateur quotidien qui..."
        - Contextes médicaux : "Mon médecin m'a prescrit..."
        - Environnements spécifiques : "Je dois utiliser l'équipement dans..."
        - Besoins particuliers : "J'ai des difficultés avec..."
        - Scenarios d'assistance : "Mon aidant doit pouvoir..."

    3. Contenu
        - Couvrir tous les aspects : spécifications, fonctionnalités, options, entretien
        - Varier les contextes d'utilisation et les profils utilisateurs
        - Inclure des questions de suivi logiques
        - Créer entre 100 et 500 entrées uniques

    4. Qualité des données
        - Toutes les entrées en français
        - Réponses adaptées au contexte de l'input
        - Éviter les répétitions
        - Style naturel et professionnel

    Exemples du format attendu :
    [
        {{
            "instruction": "Comment ajuster la hauteur du siège ?",
            "input": "Je suis un kinésithérapeute qui doit adapter le fauteuil pour différents patients au cours de la journée, et j'ai besoin d'un réglage rapide et précis.",
            "output": "Pour un ajustement rapide entre patients : 1) Utilisez le levier à dégagement rapide situé sous le siège 2) Réglez à la hauteur souhaitée - l'échelle graduée permet un réglage précis 3) Relâchez le levier pour verrouiller. Le système permet un ajustement en moins de 30 secondes."
        }},
        {{
            "instruction": "Quelle est l'autonomie du TDX SP2 ?",
            "input": "Je suis commercial et je me déplace toute la journée en ville pour visiter des clients. J'ai besoin de savoir si l'autonomie sera suffisante pour ma journée de travail de 8h.",
            "output": "L'autonomie de 31 km est adaptée à une journée de travail en ville. Pour votre usage professionnel, cela permet environ 6-7h de déplacements continus avec des arrêts chez les clients. Prévoyez une recharge nocturne pour maintenir la performance."
        }},
        {{
            "instruction": "Comment nettoyer le fauteuil ?",
            "input": "Je travaille dans un EHPAD où le fauteuil est utilisé par plusieurs résidents, et nous devons respecter des protocoles d'hygiène stricts entre chaque utilisation.",
            "output": "Pour un nettoyage conforme aux normes EHPAD : 1) Utilisez un désinfectant hospitalier homologué 2) Insistez sur les accoudoirs et la manette de contrôle 3) Essuyez avec un chiffon microfibre 4) Laissez sécher 5 minutes avant réutilisation. Évitez les jets d'eau directs sur les composants électroniques."
        }}
    ]
    '''

    return prompt_template.format(text=text)

In [166]:
def convert_to_json(text):
    try:
        text = text.replace("json", "").replace("```", "")
        return json.loads(text)
    except Exception as e:
        print("Erreur lors de la conversion en JSON:", e)
        return []

In [167]:
from dotenv import load_dotenv
load_dotenv()

# Configuration of the Gemini model
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")

def generate_data(prompt: str) -> list:
    try:
        response = model.generate_content(prompt)
        return convert_to_json(response.text)
    except Exception as e:
        print("Erreur:", e)
        return generate_data(prompt)

## Création du data set

In [168]:
dataset = []
for pdf_file in pdf_files:
    print(f"Traitement du fichier : {pdf_file.split('/')[-1]}")
    data_generate = generate_data(prompt=get_prompt(read_pdf(pdf_file)))
    dataset.extend(data_generate)
    print(f"Data size -> {len(dataset)}\n")

Traitement du fichier : AVIVA RX40 - MU.pdf
Data size -> 57

Traitement du fichier : AVIVA FX40 - Brochure FR.pdf
Data size -> 103

Traitement du fichier : Storm 4 - MM.pdf
Erreur lors de la conversion en JSON: Unterminated string starting at: line 351 column 15 (char 32207)
Data size -> 103

Traitement du fichier : TDX - MU.pdf
Data size -> 160

Traitement du fichier : AVIVA RX40 - Brochure FR.pdf
Erreur lors de la conversion en JSON: Unterminated string starting at: line 406 column 15 (char 30343)
Data size -> 160

Traitement du fichier : M5 Corpus - BC.pdf
Data size -> 229

Traitement du fichier : AVIVA RX40 - BC - AA2.pdf
Erreur lors de la conversion en JSON: Unterminated string starting at: line 365 column 14 (char 26132)
Data size -> 229

Traitement du fichier : Sango - Brochure.pdf
Data size -> 268

Traitement du fichier : AVIVA FX40 - BC.pdf
Erreur lors de la conversion en JSON: Expecting property name enclosed in double quotes: line 156 column 100 (char 9977)
Data size -> 268


In [169]:
output_file = "data/faq.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=4, ensure_ascii=False)

print(f"Données sauvegardées dans {output_file}")

Données sauvegardées dans data/faq.json
